# Experiment Tracking
## Introduction
> __MLFlow offers an experiment-tracking component in the form of an API and a UI. This component enables the logging and visualisation of experimental data.__

## Benefits
With this component, it is possible to log the following:
- model parameters
- code versions (git commit hashes)
- metrics
- generated artifacts

__`MLFlow` tracking is organised around runs, which are simply an approach for executing programs__.

`mlflow` records each run in 
- the local files, 
- an SQLAlchemy database, or 
- remote storage (via the [`mlflow.set_tracking_uri()`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.set_tracking_uri) function).

For more information on storage, check out the documentation [here](https://mlflow.org/docs/latest/tracking.html#how-runs-and-artifacts-are-recorded).

> __Via `MLFlow`, we can track, version and create comprehensive experiments, starting with ETL and ending with deployment.__

## Important Concepts
There are a few main concepts to understand when using MLFlow.
- __experiment__: mainly [`mlflow.set_experiment(UNIQUE_NAME_OF_EXPERIMENT)`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.set_experiment), which sets the current experiments and optionally creates them if they do not exist.
- __run__: an experiment can consist of multiple single runs. Context manager [`mlflow.start_run()`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.start_run).
- __logging__: essentially logging data from an experiment. Here are the related functions:
    - [`mlflow.log_param(key, value)`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_param) logs hyperparameters and other settable parameters under the current run.
    - [`mlflow.log_metric(key, value)`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_metric).
    - [`mlflow.log_artifact(local_path)`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_artifact) logs the created file(s) (e.g. models, generated text, etc.) under the current run.
    
Next, as a demonstration, we run and log a __non-flavoured__ (i.e. without specific integrations) dummy experiment:

In [14]:
import mlflow


def create_dummy_file():
    features = "rooms, zipcode, median_price, school_rating, transport"
    with open("features.txt", "w") as f:
        f.write(features)


create_dummy_file()

# Create experiment (artifact_location=./ml_runs by default)
mlflow.set_experiment("Dummy Experiments")

# By default experiment we've set will be used
with mlflow.start_run():
    mlflow.log_artifact("features.txt")
    mlflow.log_param("learning_rate", 0.01)
    for i in range(10):
        mlflow.log_metric("Iteration", i, step=i)

To visualise and explore the saved data, we run the `mlflow ui` command and visit [`http://localhost:5000 `](http://localhost:5000) in a web browser (__the data will be saved in `./mlruns`__).

Run the following in the terminal:

In [8]:
# !mlflow ui --help

After navigating to the experiment, we can observe the `Iteration` being logged as shown below:

![](images/mlflow_ui.png)

## Model Format

> MLFlow provides a standard format for saving ML models (from various libraries), which aids the usage of models (e.g. inference on REST API, cloud, etc.). 

MLFlow models consist of the following:
- a directory with arbitrary files defined by the model.
- an `MLmodel` file (written in yaml) that specifies the contents of the model.

Below, we demonstrate how to save a model (in this case, `sklearn`) in Python:

In [9]:
mlflow.sklearn.save_model(model, "my_model")

NameError: name 'model' is not defined

The above code creates the following directory in our `cwd`:

```bash
my_model/
├── MLmodel
└── model.pkl
```

The contents of the `MLModel` are equally easy to understand:

```yml
---
time_created: 2021-04-03T17:28:53.35

flavours:
  sklearn:
    sklearn_version: 0.24.1
    pickled_model: model.pkl
  python_function:
    loader_module: mlflow.sklearn
```

### Model signature

To deploy and occasionally run a model (e.g. in `tensorflow`), the __model signature__ must be specified.

> __Model signature specifies the type and shape of inputs going through the model.__

*Things to note*
- Standard casting rules apply (upcasting is fine; however, downcasting will raise an error).
- The model signature aids the reading of inputs when they are sent in JSON format via REST API or the like.

It can be added to the `MLModel` file via any of the two modes: column signature or tensor signature.

#### Column signature

> In this mode, the user specifies the input signature by specifying every possible column input.

This mode is supported by all flavours (frameworks); however, it might be difficult to achieve in some cases.

Consider the example below, which uses the `iris` dataset:

```yaml
signature:
    inputs: '[{"name": "sepal length (cm)", "type": "double"}, {"name": "sepal width
      (cm)", "type": "double"}, {"name": "petal length (cm)", "type": "double"}, {"name":
      "petal width (cm)", "type": "double"}]'
    outputs: '[{"type": "integer"}]'
```

#### Tensor signature

> In this mode, the user specifies the input for deep-learning inputs (e.g. images) via the tensor shape.

Consider the image-oriented example:

```yaml
signature:
    inputs: '[{"name": "images", "dtype": "uint8", "shape": [-1, 28, 28, 1]}]'
    outputs: '[{"shape": [-1, 10], "dtype": "float32"}]'
```

#### Inferring input shapes

Generally, it is relatively easy (and less error-prone) to infer `dtype` and the shape through our code. These can easily be achieved via [`mlflow.models.infer_signature`](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.infer_signature).

An example is provided below.

In [11]:
import pandas as pd
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

iris = datasets.load_iris()
iris_train = pd.DataFrame(iris.data, columns=iris.feature_names)
clf = RandomForestClassifier(max_depth=7, random_state=0)
clf.fit(iris_train, iris.target)
signature = infer_signature(iris_train, clf.predict(iris_train))

mlflow.sklearn.log_model(clf, "iris_rf", signature=signature)

For `infer_signature`,
- pass the input data (conventionally `torch.Tensor`, `pd.DataFrame`, `np.ndarray` or other standard types).
- pass data through the model as the second argument, which will create `outputs` automatically.


The `mlflow.sklearn.log_model` command saves the model to the file in the `cwd` named `iris_rf` with the specified signature.
Note that we can load it later from the disk (__it must be customised to the flavour in which it was saved__).

In [ ]:
# Load sklearn model

sklearn_model = mlflow.sklearn.load_model("iris_rf")

## Conclusion
At this point, you should have a good understanding of 
- MLFlow experiment tracking and the important associated concepts.
- model formats.